## 변수 영역과 클로저의 상호작용 방식을 이해하라

숫자로 이뤄진 list 를 정렬하되, 정렬한 리스트의 앞쪽에는 우선순위를 부여한 몇몇 숫자를 위치시켜야 한다고 가정하자.
- 이 패턴은 사용자 인터페이스를 표시하면서 중요한 메시지나 예외적인 이벤트를 다른 것보다 우선해 표시하고 싶을 때 유용하다.
- 일반적인 방법은 sort 메서드에 key 인자로 도우미 함수를 전달하는 것이다.
- list 는 각 원소를 정렬할 때 이 도우미 함수가 반환되는 값을 기준으로 사용한다.

In [1]:
def sort_priority(values, group):
    def helper(x):
        if x in group:
            return (0, x)
        return (1, x)
    
    values.sort(key=helper)

In [2]:
numbers = [8, 3, 1, 2, 5, 4, 7, 6]
group = {2, 3, 5, 7}

sort_priority(numbers, group)
print(numbers)

[2, 3, 5, 7, 1, 4, 6, 8]


이 함수가 예상대로 작동하는 세 가지 이유가 있다.
- 파이썬이 클로저(closure)를 지원 : 클로저란 자신이 정의된 영역 밖의 변수를 참조하는 함수다. 클로저로 인해 도우미 함수가 sort_priority 함수의 group 인자에 접근할 수 있다.
- 파이썬에서 함수가 일급 시민(first-class citizen) 객체임 : 일급 시민 객체라는 말은 이를 직접 가리킬 수 있고, 변수에 대입하거나 다른 함수에 인자로 전달할 수 있으며, 식이나 if 문에서 함수를 비교하거나 함수에서 반환하는 것 등이 가능하다는 것을 의미한다. 이 성질로 sort 메서드는 클로저 함수를 key 인자로 받을 수 있다.
- 파이썬에서는 시퀀스(튜플 포함)를 비교하는 구체적인 규칙이 있음: 파이썬은 시퀀스를 비교할 때 0번 인덱스에 있는 값을 비교한 다음, 이 값이 같으면 다시 1번 인덱스에 있는 값을 비교한다. 이런 식으로 순서대로 원소를 비교해 두 값이 같으면 그 다음 원소로 넘어가는 작업을 시퀀스의 모든 원소를 다 비교하거나 결과가 정해질 때까지 계속한다. 이로 인해 helper 클로저가 반환하는 튜플이 서로 다른 두 그룹을 정렬하는 기준 역할을 할 수 있다.

이 함수가 우선순위가 높은 원소가 있는지 여부도 반환하게 만들어서 UI가 우선순위가 높은 원소가 있을 떄와 아닌 때를 구분해 처리할 수 있다면 더 좋을 것이다.

이미 각 원소가 어떤 그룹게 들어 있는지 결정하는 클로저 함수가 있으므로, 이 클로저를 사용해 우선순위가 높은 우너소를 발견했음을 표시하는 플래그를 설정하면 어떨까?

In [9]:
def sort_priority2(numbers, group):
    found = False

    def helper(x):
        if x in group:
            found = True
            return (0, x)
        return (1, x)
        
    numbers.sort(key=helper)
    return found
    
found = sort_priority2(numbers, group)
print(found)
print(numbers)

False
[2, 3, 5, 7, 1, 4, 6, 8]


왜 found 는 False 가 나왔을까? True여야 하는데...

파이썬은 변수를 참조할 때 다음 순서로 영역을 뒤진다.
- 현재 함수의 영역
- 현재 함수를 둘러싼 영역(현재 함수를 둘러싸고 있는 함수 등)
- 현재 코드가 들어 있는 모듈의 영역(전역 영역(global scope))이라고도 부름)
- 내장 영역(built-in scope)

식이 참조하는 이름에 해당하는 변수가 이 네 가지 영역에 없으면 NameError 예외가 발생한다.

변수에 값을 대입하는 것은 다른 방식으로 작동한다.
- 변수가 현재 영역에 이미 정의돼 있다면 그 변수의 값만 새로운 값으로 바뀐다.
- 하지만 변수가 현재 영역에 정의돼 있지 않다면 파이썬은 변수 대입을 변수 정의로 취급한다.
- 결정적으로 이렇게 새로 정의된 변수의 영역은 해당 대입문이나 식이 들어 있던 함수가 된다.

따라서 helper 함수의 클로저 안에서 이 대입문은 helper 영역 안에 새로운 변수를 정의하는 것으로 취급되지, sort_priority2 안에서 기존 변수에 값을 대입하는 것으로 취급되지는 않는다.

이 문제는 영역 지정 버그(scoping bug) 라고 부르기도 한다.
- 하지만 이 동작은 의도에 따른 결과다.
- 의도는 함수에서 사용한 지역 변수가 그 함수를 포함하고 있는 모듈 영역을 더럽히지 못하게 막는 것이다.
- 이런 식으로 처리하지 않으면 함수 내에서 사용한 모든 대입문이 전역 모듈 영역에 쓰레기 변수를 추가하게 된다. 그 결과 추가된 불필요한 변수들로 인해 잡음이 늘어날 뿐 아니라 추가된 전역 변수와 클로저의 상호작용에 의해 알아내기 힘든 상황이 생길 수 있다.

파이썬에서는 클로저 밖으로 데이터를 끌어내는 특별한 구문이 있다. nonlocal 문을 이용하면 된다.
- 한계점은 모듈 수준 영역까지 변수 이름을 찾아 올라가지 않는다는 것 뿐이다.

In [10]:
def sort_priority2(numbers, group):
    found = False

    def helper(x):
        nonlocal found
        
        if x in group:
            found = True
            return (0, x)
        return (1, x)
        
    numbers.sort(key=helper)
    return found

nonlocal 문은 대입할 데이터가 클로저 밖에 있어서 다른 영역에 속한다는 사실을 분명히 알려준다.

이 문장은 변수 대입 시 직접 모듈 영역을 사용해야 한다고 지정하는 global 문을 보완해준다.

하지만 가급적 nonlocal 도 안 쓰는 걸 권장한다.

In [15]:
class Sorter:
    def __init__(self, group):
        self.group = group
        self.found = False        
    
    def __call__(self, x):
        if x in self.group:
            self.found = True
        
            return (0, x)
        return (1, x)
    
sorter = Sorter(group)
numbers.sort(key=sorter)
assert sorter.found is True

다소 길지만 이렇게 하는게 읽기는 더 쉽다.
- `__call__` 에 대해서는 추후 학습